<a><img src = "barbecue-bbq-dinner-111131.jpg" width =600, height = 100 align = "center"></a>


<h1 align=center><font size = 5 color = brown>Predicting Cuisines having Maximum Potential in a Neighbourhood Restaurant</font></h1>

## Table of contents <a name="ToC"></a>
* [Business Problem](#introduction)
* [Description of Data](#data)
* [Methodology](#methodology)
* [Analysis of Results and Discussion](#results)
* [Conclusion](#conclusion)

### <font color = brown><i> Business Problem</font></i> <a name="introduction"></a>
<font face=calibri size=3>The objective of this report is to help anyone who intends to set up a restaurant or eatery in a neighborhood decide on where and what type of cuisines to serve through analysis of data sourced from foursquare and US census bureau. 
<br/><br/>
“Explore where one should open a restaurant and what type of cuisine to serve through use of data analytics and Machine Learning”.
<br/><br/>
One business strategy coule be to explore cuisine which is not served in the neighbourhood abother could be venture into cuisne mostly served in the area.

</font>
[Back To ToC](#ToC)

### <font color = brown><i>  Description of Data</i></font> <a name="data"></a>
<font face=calibri size=3>
For this project I intend to use data gathered from foursquare about type of cuisine most served at a neighbourhood and US census portal about neighbourhood profile through use of API’s. 
<br/><br/>
I will focus on using supervised learning algorithms classify data points into categorical type of cuisine using best classification model
<br/><br/>
US Census has API’s which provide information about geographies for more than 18000 variables. I plan to use variables pertaining to age, race, income etc. along with data obtained from foursquare regarding number of place, type of cuisine and then predict the type of cuisine can be served when setting a restaurant in a new neighborhood.

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* Demographic profile of the neighborhood

Based on center of location I plan to get number of existing restaurants in the neighborhood in a radius of 25 or 50km and then take them as target variable based on Demographic profile.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants for each cuisine type in neighborhood i.e approximate radius of <25 km> will be obtained using **Foursquare API**
* Demographic profile of the neighborhood will be obtained using **US Census API**
<br>
<br>

[Back To ToC](#ToC)

### <font color = brown><i>  Methodology</i> </font> <a name="methodology"></a>

<font face=calibri size=3>
To predicit best type of cuisine to serve in a neighbourhood I followed following series of Step.

* Finding Demograhich profile of all US regions defined by the US Census. I would be using Income, Age and work related variables. Description of these variables is defined in US census(The list of variables I have used is given in file projParams.py. I have taken variables for Male/Femal Age, Education profile and Income distribution profile. 
<br><br>

* Downling longitude and latitude of all these locations using google API. Since foursquare API allows only 950 free calls only per day so I have taken only 1400 locations. 
<br><br>

* Getting count of restaurants and cuisine served in all the regions. Then finding the topmost cuisine served in the regions.   This topmost resturant cusisine type becomes my target variable for prediction using classification models.

<br>
* Once I have predicted cuisine type corrected I would be using regression to predict count of restaurants each region can support. With count of exisiting restaurants  we got from fourSquare I will find difference of predicted count. The regions where the difference is maximum I can conclude that is the best location for setting the Restaurant with maximum potential.

[Back To ToC](#ToC)
#### Getting all the required imports

In [1]:
#!pip install census   
#!pip install us    
#!pip install geopy

import json                               # library to handle JSON files
import time                               # library for timing a script
import numpy                   as np      # library to handle data in a vectorized manner
import pickle
import pandas                  as pd      # library for data analsysis
import requests                           # library to handle requests

import operator                           # library for Mathematial Operators
import importlib

from bs4            import BeautifulSoup  # library for parsing the html page
from time           import sleep          # 
from pandas.io.json import json_normalize # library to tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
%matplotlib inline
import matplotlib.cm     as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.utils.multiclass import unique_labels

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Reauired Libraries imported.')

Reauired Libraries imported.


In [2]:
#### Importing Project specific Constants, Lists and API Keys
import projParams as config

In [3]:
from census import Census
from us     import states

geolocator = Nominatim(user_agent="DataScieneProject")
c = Census(config.projectSecrets['clientAPIKey'],year=2017)

In [4]:
def getCoordinatesFromGoogle(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]

#### <font color = brown><i>  Getting Supporting Data </i> </font> <a name="places"></a>

<font face=calibri size=3>
<br>
[Back To ToC](#ToC)

In [5]:
USMainLandStatesDF = pd.read_csv("usMainLandStates.csv")
USMainLandStatesDF.state = USMainLandStatesDF.state.astype(int)
print('US Mainland States for which this study is limited to = {} '.format(USMainLandStatesDF.shape[0]) )

### https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html
### File downloaded from above link is formatted and converted into excel for reading by pandas
### 
allGeoCodesDF = pd.read_csv("allGeoCodes.csv")
print('Total Number of Geographic Area captured by US Census = {} '.format(allGeoCodesDF.shape[0]) )

allGeoCodesDF = pd.merge(allGeoCodesDF,
                         USMainLandStatesDF.loc[:, ['state','stateName']],
                         left_on =['StateCode_FIPS'] , 
                         right_on=['state'])

allGeoCodesDF.drop('state', 1, inplace=True)
allGeoCodesDF.head()

US Mainland States for which this study is limited to = 51 
Total Number of Geographic Area captured by US Census = 43934 


,SummaryLevel,StateCode_FIPS,County_Code_FIPS,County_Subdivision_Code_FIPS,Place_Code_FIPS,Consolidtated_City_Code_FIPS,Area_Name,stateName
0,40,1,0,0,0,0,Alabama,Alabama
1,50,1,1,0,0,0,Autauga County,Alabama
2,50,1,3,0,0,0,Baldwin County,Alabama
3,50,1,5,0,0,0,Barbour County,Alabama
4,50,1,7,0,0,0,Bibb County,Alabama


#### <font color = brown><i>  Getting Places Information from US Census Portal</i> </font> <a name="places"></a>

<font face=calibri size=3>
Places and 'County- SubDivision' are  the smallest level of Geographic region defined the US Census for which demographic information is available. 
<br>
[Back To ToC](#ToC)

In [6]:
def getPlaceDemographics():
    a = c.acs5.get('B01001_001E', {'for': 'place:*'})
    placeDemoDF = pd.DataFrame(a)
    placeDemoDF.set_index(['state', 'place'], inplace=True)  
    
    for name in varList:
        if (varList[name] != 'B01001_001E'):
            tempList = c.acs5.get(varList[name], {'for': 'place:*'})
            tempDF   = pd.DataFrame(tempList) 
            tempDF.set_index(['state', 'place'], inplace=True)                
            placeDemoDF = pd.merge(placeDemoDF, tempDF, left_index=True, right_index=True)
            print(' .', end='')        
    print(' Done', end='')  
        
    # persisting to local drive for repeated runs/use
    placeDemoDF['county']=0
    placeDemoDF['county subdivision']=0

    placeDemoDF = pd.merge(placeDemoDF,
                           allGeoCodesDF.loc[(allGeoCodesDF["County_Code_FIPS"]             ==0) &
                                             (allGeoCodesDF["County_Subdivision_Code_FIPS"] ==0) &
                                             (allGeoCodesDF["Place_Code_FIPS"]              > 0),['StateCode_FIPS','Place_Code_FIPS','stateName','Area_Name']],
                           left_on =['state'         ,'place'] , 
                           right_on=['StateCode_FIPS','Place_Code_FIPS'])

    placeDemoDF.drop('StateCode_FIPS', 1, inplace=True)
    placeDemoDF.drop('Place_Code_FIPS',1, inplace=True)    
    placeDemoDF.to_csv("placeDemoDF.csv", index=True)

    return placeDemoDF

loaded = False
try:
    placeDemoDF = pd.read_csv("placeDemoDF.csv", encoding="cp1252", dtype= config.varListTypes)
    loaded = True
except:
    pass

print('Status of Place Demographics file                     = {} '.format(loaded) )
# If load failed then call US Census and populate variables 
if not loaded:    
    placeDemoDF = getPlaceDemographics()    
print('Total Number of Places Demographic profile available  = {} '.format(placeDemoDF.shape[0]) )

placeDemoDF.head()

Status of Place Demographics file                     = True 
Total Number of Places Demographic profile available  = 19506 


,state,place,B01001_001E,B01002_001E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,...,B19001_017E,B23022_003E,B23022_027E,B25008_003E,B25031_001E,B25077_001E,county,county subdivision,stateName,Area_Name
0,1,124,2594.0,44.9,113.0,106.0,58.0,69.0,40.0,105.0,...,21.0,543.0,384.0,771.0,553.0,80200.0,0,0,Alabama,Abbeville city
1,1,460,4404.0,42.7,63.0,106.0,200.0,136.0,90.0,147.0,...,14.0,928.0,903.0,1083.0,862.0,97100.0,0,0,Alabama,Adamsville city
2,1,484,725.0,47.6,14.0,27.0,19.0,19.0,36.0,23.0,...,0.0,185.0,108.0,221.0,356.0,75100.0,0,0,Alabama,Addison town
3,1,676,318.0,40.3,0.0,10.0,5.0,4.0,3.0,24.0,...,0.0,48.0,55.0,114.0,533.0,38800.0,0,0,Alabama,Akron town
4,1,820,32567.0,37.0,593.0,990.0,1274.0,1326.0,1252.0,1404.0,...,293.0,8958.0,7285.0,5648.0,983.0,167800.0,0,0,Alabama,Alabaster city


#### <font color = brown><i>  Getting County - SubDivision Information from US Census Portal</i> </font> <a name="subdiv"></a>
<font face=calibri size=3>
Places and 'County- SubDivision' are  the smallest level of Geographic region defined the US Census for which demographic information is available. 
<br>
[Back To ToC](#ToC)

In [7]:
def getCountySubdivDemographics():
    USMainLandStatesList = list(USMainLandStatesDF.state)
    stateCountySubdivDF = pd.DataFrame(columns=config.stateCountySubdivCols)

    for st in USMainLandStatesList:
        st= str(st).zfill(2)
        print('Included State : {}  Variable :{} '.format(st , 'B01001_001E')) 
        b = c.acs5.get('B01001_001E', {'for': 'county subdivision:*', 'in': 'state:'+ st }  )
        countySubdivDF = pd.DataFrame(b)
        countySubdivDF.set_index(['state', 'county','county subdivision'], inplace=True)
        
        for name in varList:
            if (varList[name] !='B01001_001E'):
                print('Included State : {}  Variable :{} '.format(st , varList[name])) 
                tempList = c.acs5.get(varList[name], {'for': 'county subdivision:*', 'in': 'state:'+ st})
                tempDF   = pd.DataFrame(tempList)
                tempDF.set_index(['state', 'county','county subdivision'], inplace=True)                
                countySubdivDF = pd.merge(countySubdivDF, tempDF, left_index=True, right_index=True)
                print(' .', end='')
    
    stateCountySubdivDF =  stateCountySubdivDF.append(countySubdivDF)
    print(' Done', end='') 

    ### 
    stateCountySubdivDF['place']=0    
    stateCountySubdivDF = pd.merge(stateCountySubdivDF,
                 allGeoCodesDF.loc[(allGeoCodesDF["County_Code_FIPS"]             > 0) &
                                   (allGeoCodesDF["County_Subdivision_Code_FIPS"] > 0) &
                                   (allGeoCodesDF["Place_Code_FIPS"]              ==0 ),['StateCode_FIPS','County_Code_FIPS','County_Subdivision_Code_FIPS','stateName','Area_Name']],
                           left_on =['state'         ,'county'          ,'county subdivision'] , 
                           right_on=['StateCode_FIPS','County_Code_FIPS','County_Subdivision_Code_FIPS'])

    stateCountySubdivDF.drop('StateCode_FIPS', 1, inplace=True)
    stateCountySubdivDF.drop('County_Code_FIPS',1, inplace=True)
    stateCountySubdivDF.drop('County_Subdivision_Code_FIPS',1, inplace=True)
    
    return stateCountySubdivDF
    # persisting to local drive for repeated runs/use
    stateCountySubdivDF.to_csv("stateCountySubdivDF.csv")

loaded = False
try:
    stateCountySubdivDF = pd.read_csv("stateCountySubdivDF.csv", encoding="cp1252", dtype= config.varListTypes)
    loaded = True
except:
    pass
     
print('Status of County Sub Div Demographics file            = {} '.format(loaded) )    
# If load failed then call US Census and populate variables 
if not loaded:    
    stateCountySubdivDF = getCountySubdivDemographics()
print('Total Numbe of County Sub Div Demographics available  = {} '.format(stateCountySubdivDF.shape[0] ) )
stateCountySubdivDF.head() 

Status of County Sub Div Demographics file            = True 
Total Numbe of County Sub Div Demographics available  = 21137 


,state,county,county subdivision,B01001_001E,B01002_001E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,...,B19001_016E,B19001_017E,B23022_003E,B23022_027E,B25008_003E,B25031_001E,B25077_001E,place,stateName,Area_Name
0,9,1,63480,9274.0,47.1,141.0,257.0,103.0,143.0,464.0,...,474.0,1084.0,2376.0,2162.0,1346.0,1586.0,597500.0,0,Connecticut,Redding town
1,9,1,74190,52529.0,44.1,1438.0,1394.0,1468.0,1561.0,1865.0,...,1894.0,1438.0,13189.0,13763.0,9039.0,1224.0,250700.0,0,Connecticut,Stratford town
2,9,1,73070,128851.0,37.0,6045.0,5291.0,4973.0,4239.0,4699.0,...,4366.0,9319.0,38568.0,33626.0,57359.0,1704.0,516000.0,0,Connecticut,Stamford town
3,9,1,50580,20357.0,43.2,162.0,92.0,658.0,591.0,854.0,...,730.0,3140.0,4686.0,3658.0,3679.0,2001.0,1439600.0,0,Connecticut,New Canaan town
4,9,1,50860,14091.0,45.1,293.0,259.0,355.0,365.0,627.0,...,647.0,743.0,3830.0,3631.0,934.0,2180.0,356600.0,0,Connecticut,New Fairfield town


#### <font color = brown><i> Creating Places of Interest </i> </font> <a name="locations"></a>
<font face=calibri size=3>

I will be merging 'places' dataframe and 'County-SubDivision' dataframe in a single dataframe as Locations of interest. 

For each of these locations of interest I will be finding the count of different type of cuisines served in a restaurant. 

I have grouped various type of restaurant by their category into broad 12 cuisines types (https://en.wikipedia.org/wiki/List_of_cuisines) of the World based. Although this may be debatable and may not be accurate but for purpose of this report I am not diving deeper into this.

If a cuisine type has very sub-catregories I have merged it to top leval or cuisine of region closest to its region. 

<br>
[Back To ToC](#ToC)

In [8]:
locationsDF = placeDemoDF.append(stateCountySubdivDF)
locationsDF['locationsName'] = locationsDF['Area_Name']+', '+ locationsDF['stateName'] + ', USA'
locationsDF.to_csv("locationsDF.csv", index=False) 

print('Total Number of County-Sub Div from US Census.........= {} '.format(stateCountySubdivDF.shape[0] ) )
print('Total Places of Interest from US Census...............= {} '.format(placeDemoDF.shape[0] ) )
print('Total Number of Places of Interest for evaluation.....= {} '.format(locationsDF.shape[0] ) )


Total Number of County-Sub Div from US Census.........= 21137 
Total Places of Interest from US Census...............= 19506 
Total Number of Places of Interest for evaluation.....= 40643 


#### <font color = brown><i> Getting Geographic Co-ordinates of places of Interest </i> </font> <a name="geolocations"></a>
<font face=calibri size=3>

<br>
[Back To ToC](#ToC)

In [9]:
def getLocationCoordinates(locationNamesList):
    counter      = 0
    step         = 1000
    placeGeoCord = []
    dfSize       = len(locationNamesList)
    while counter < dfSize :
        if (counter + step < dfSize ):
            batch = locationNamesList[counter:counter+step]        
        else:
            batch = locationNamesList[counter:]
        counter = counter + step

        batchCoord = [getCoordinatesFromGoogle(config.projectSecrets['googleAPIkey'], x) for x in batch]
        placeGeoCord = placeGeoCord + batchCoord

        print(' .', end='')
    print(' Done')
    return placeGeoCord
    
loaded = False
try:
    locationsSampleDF = pd.read_csv("locationsSampleDF.csv", encoding="cp1252", dtype= config.varListTypes)
    loaded = True
except:
    pass

print('Is sampled Places of Interest file available..........= {} '.format(loaded ))

## If load failed then calling google API's and getting data only for sample of 20% of Locations Data
## which is about 8,000 records.
##
if not loaded:
    locationsSampleDF = locationsDF.sample(frac = 0.20 )    
    placeGeoCord = getLocationCoordinates( list(locationsSampleDF['locationsName']) )
    
    locationsSampleDF['lat'] = [x[0] for x in placeGeoCord]
    locationsSampleDF['long']= [x[1] for x in placeGeoCord]
    
    # persisting to local drive for repeated runs/use
    locationsSampleDF.to_csv("locationsSampleDF.csv", index=False)

locationsSampleDF = locationsSampleDF[locationsSampleDF['long'].notnull()]

print('Tot Num of Sampled Geo-Coordinated Places of Interest.= {} '.format(locationsSampleDF.shape[0] ) )  
locationsSampleDF.head()

Is sampled Places of Interest file available..........= True 
Tot Num of Sampled Geo-Coordinated Places of Interest.= 1409 


,Area_Name,B01001_001E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,...,B25031_001E,B25077_001E,county,county subdivision,place,state,stateName,locationsName,lat,long
0,Brownsville borough,2181.0,46.0,60.0,111.0,43.0,90.0,59.0,72.0,62.0,...,544.0,58200.0,51,9432,0,42,Pennsylvania,"Brownsville borough, Pennsylvania, USA",40.023685,-79.883936
1,Jackson township,1044.0,10.0,27.0,13.0,35.0,26.0,48.0,45.0,1.0,...,707.0,112500.0,111,34825,0,20,Kansas,"Jackson township, Kansas, USA",39.032697,-96.556988
2,Dinosaur town,315.0,30.0,21.0,20.0,2.0,5.0,13.0,2.0,12.0,...,821.0,101900.0,0,0,20495,8,Colorado,"Dinosaur town, Colorado, USA",40.243578,-109.014561
3,Hobgood town,353.0,13.0,13.0,7.0,4.0,17.0,22.0,19.0,6.0,...,748.0,59000.0,0,0,31860,37,North Carolina,"Hobgood town, North Carolina, USA",36.029879,-77.397746
4,Marceline city,2107.0,40.0,53.0,100.0,56.0,37.0,75.0,56.0,36.0,...,486.0,60800.0,0,0,45866,29,Missouri,"Marceline city, Missouri, USA",39.711970,-92.948253


### Foursquare
Foursquare categorizes reastaurancts and places on basis of type of cuisine it serves. Based on geographical region restaurancts can be classifed in to broad place of origin.

For this project I am only interest in venues in 'food category' I will use FourSquare to get predominant cusinene served in the neighbourhood restaurants.

In [10]:
def getPreferredCuisine(venues, rank):
    locationCuisineCount = {'eastAsianCuisine'       : 0,
                            'southAsianCuisine'      : 0,
                            'southEastAsianCuisine'  : 0,
                            'westAsianCuisine'       : 0,
                            'africanCuisine'         : 0,
                            'centralEuroCuisine'     : 0,
                            'eastNorthernEuroCuisine': 0,
                            'southernEuroCuisine'    : 0,
                            'westernEuroCuisine'     : 0,
                            'northAmericanCuisine'   : 0,
                            'southAmericanCuisine'   : 0,
                            'caribbeanCuisine'       : 0,
                            'Others':0}    
    for venue in venues:
        locationCuisineCount[venue[1]] = (locationCuisineCount[venue[1]] +1)
    
    ### Removing Others form Dictionary since Others include large number of values in Dataset
    if 'Others' in locationCuisineCount.keys():     
        locationCuisineCount.pop('Others')

    ## Getting cusine which has maximum number of restaurents in a location
    sorted_x = sorted(locationCuisineCount.items(), key=operator.itemgetter(1))
    
    if rank == 1:
        preferredCusine      = sorted_x[len(sorted_x)-1][0]
        preferredCusineCount = sorted_x[len(sorted_x)-1][1]
    if rank == 2:
        preferredCusine      = sorted_x[len(sorted_x)-2][0]
        preferredCusineCount = sorted_x[len(sorted_x)-2][1]
    if rank ==3:
        preferredCusine      = sorted_x[len(sorted_x)-3][0]
        preferredCusineCount = sorted_x[len(sorted_x)-3][1]

    return preferredCusine, preferredCusineCount
                
def getCatType(categories):
    try:
        for cat in categories:
            category =  config.cuisineType[cat['id']]
    except:
        category = "Others"
    return category

def parseResults(results):
    venues = [(item['venue']['name'],
               getCatType(item['venue']['categories']),
               item['venue']['location']['distance']) for item in results]
    return venues

def getVenues(lat, lon):
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(config.projectSecrets['4SquareClientID'], config.projectSecrets['4SquareclientSecret'], config.version, lat,  lon, config.food_category , config.radius, config.limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
    except:
        print("Err",end='')
        results = None
    return  results

In [11]:
def getLocationStatistics(lat, lon):
    counter = 0
    start = time.time()
    locationStat=[]    
    for lat, lon in zip(lat, lon):
        results    =  getVenues(lat , lon) 
        sleep(.5)    ### Introducing time delay of .5 sec, since fourSquare allows 2 API calls per second.
        if results is not None:
            venues      = parseResults(results) 
            totalVenues = len(venues)
            top1CuisineNM, top1CuisineCnt = getPreferredCuisine(venues, 1)
            top2CuisineNM, top2CuisineCnt = getPreferredCuisine(venues, 2)
            top3CuisineNM, top3CuisineCnt = getPreferredCuisine(venues, 3)
        else:
            totalVenues   = most_pop_cuisine_cnt = second_popular_cuisine_count = third_popular_cuisine_count = 0
            top1CuisineNM = top2CuisineNM        = top3CuisineNM                = np.nan
            
        locatStat= {'region_cnt'            : totalVenues,
                    'most_pop_cuisine_nm'   : top1CuisineNM, 
                    'most_pop_cuisine_cnt'  : top1CuisineCnt, 
                    'second_pop_cuisine_nm' : top2CuisineNM,
                    'second_pop_cuisine_cnt': top2CuisineCnt,                      
                    'third_pop_cuisine_nm'  : top3CuisineNM,
                    'third_pop_cuisine_cnt' : top3CuisineCnt} 
        locationStat.append(locatStat)
        print(' .', end='')
    print(' done.')
    end = time.time()
    print(end - start)
    return locationStat

# First try to load from local file system in case
loaded = False
try:
    locationsSampleStatsDF = pd.read_csv("locationsSampleFourSquareStatsDF.csv", encoding="cp1252", dtype= config.varListTypes)
    loaded = True
except:
    pass

print('Is Enriched FourSquare file with restaurant Info......= {} '.format(loaded ))

# If load failed use the Foursquare API to get the data
if not loaded:    
    lats  = list(locationsSampleDF.lat)
    longs = list(locationsSampleDF.long)
        
    #restStats   = getLocationStatistics(lats, longs)
    restStatsDF = pd.DataFrame(restStats)
    
    locationsSampleStatsDF      = pd.concat([locationsSampleDF,restStatsDF ], axis=1)
    
    # persisting to local drive for repeated runs/use
    locationsSampleStatsDF.to_csv("locationsSampleFourSquareStatsDF.csv", index=False)
    
print('Total Num of rows in enriched sample file ............= {} '.format(locationsSampleStatsDF.shape[0] ) )          

Is Enriched FourSquare file with restaurant Info......= True 
Total Num of rows in enriched sample file ............= 1394 


In [12]:
locationsSampleStatsDF.drop(['Area_Name','county','county subdivision','place','state','stateName','lat','long'], 1 , inplace=True)
locationsSampleStatsDF.head()

,B01001_001E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,...,B25031_001E,B25077_001E,locationsName,most_pop_cuisine_cnt,most_pop_cuisine_nm,region_cnt,second_pop_cuisine_cnt,second_pop_cuisine_nm,third_pop_cuisine_cnt,third_pop_cuisine_nm
0,2181.0,46.0,60.0,111.0,43.0,90.0,59.0,72.0,62.0,33.0,...,544.0,58200.0,"Brownsville borough, Pennsylvania, USA",20,northAmericanCuisine,100,11,southernEuroCuisine,8,eastAsianCuisine
1,1044.0,10.0,27.0,13.0,35.0,26.0,48.0,45.0,1.0,27.0,...,707.0,112500.0,"Jackson township, Kansas, USA",13,southAmericanCuisine,100,11,eastAsianCuisine,7,northAmericanCuisine
2,315.0,30.0,21.0,20.0,2.0,5.0,13.0,2.0,12.0,0.0,...,821.0,101900.0,"Dinosaur town, Colorado, USA",1,southAmericanCuisine,3,1,northAmericanCuisine,0,caribbeanCuisine
3,353.0,13.0,13.0,7.0,4.0,17.0,22.0,19.0,6.0,7.0,...,748.0,59000.0,"Hobgood town, North Carolina, USA",2,northAmericanCuisine,34,2,southernEuroCuisine,1,southAmericanCuisine
4,2107.0,40.0,53.0,100.0,56.0,37.0,75.0,56.0,36.0,38.0,...,486.0,60800.0,"Marceline city, Missouri, USA",2,northAmericanCuisine,17,1,southAmericanCuisine,1,eastAsianCuisine


### <font color = brown><i> Distribution of Top 3 Cuisine Choices in a Place of Interest </i> </font> 

<font face=calibri size=3 color='brown'>

For Purpose of this Report I would focusing on which alternative cuisine has maximum potential in the place of Interest. 

* I hypothesize top choice for cuisine will always caters to cuisine which matches traditional cuisine of the Region.

* Therefore I would be analysing which alternative cusine will find most acceptance in the region therefore for this purpose I will be focussing on third Most popular Choice. 

* Further given the presence of restaurnts with specific cusine I would find which alternative cusine has maximum potential in a  region.

In [13]:
print(locationsSampleStatsDF['most_pop_cuisine_nm'].value_counts(),   end="\n\n")
print(locationsSampleStatsDF['second_pop_cuisine_nm'].value_counts(), end="\n\n")
print(locationsSampleStatsDF['third_pop_cuisine_nm'].value_counts(),  end="\n\n")

northAmericanCuisine     909
southAmericanCuisine     280
southernEuroCuisine       95
caribbeanCuisine          72
eastAsianCuisine          34
southEastAsianCuisine      2
centralEuroCuisine         1
southAsianCuisine          1
Name: most_pop_cuisine_nm, dtype: int64

southAmericanCuisine     507
northAmericanCuisine     269
eastAsianCuisine         259
southernEuroCuisine      203
caribbeanCuisine         143
centralEuroCuisine         6
southEastAsianCuisine      3
westernEuroCuisine         3
southAsianCuisine          1
Name: second_pop_cuisine_nm, dtype: int64

eastAsianCuisine           464
southAmericanCuisine       346
southernEuroCuisine        220
northAmericanCuisine       172
caribbeanCuisine           155
southEastAsianCuisine       17
centralEuroCuisine           8
westernEuroCuisine           7
westAsianCuisine             2
eastNorthernEuroCuisine      2
southAsianCuisine            1
Name: third_pop_cuisine_nm, dtype: int64



<font face=calibri size=3, color ='brown'>

I observe for from above distribution that 3rd Choice of cuisnes 'eastNorthernEuroCuisine', 'westAsianCuisine', 'southAsianCuisine' has very few regions there fore I remove them from analysis. Classifications algorithms with very sparse target variable may not work well

### <font color = brown><i>  Data Cleansing & Preprocessing </i> </font> <a name="cleansing"></a>

* Removing Inconsisten Values
   It is observed in US census data NUll values are denoted by -666666666, therefore removing those values by null for
<br><br>
* Replacing NaN values wirh Median, Since Median is better representation of population data and is unaffected by extreme    values.
<br><br>
* Scaling and doing data normalization of Numeric values

<br>

[Back To ToC](#ToC)

In [14]:
locationsSampleStatsDF = locationsSampleStatsDF[(locationsSampleStatsDF.third_pop_cuisine_nm != 'eastNorthernEuroCuisine') &
                            (locationsSampleStatsDF.third_pop_cuisine_nm != 'westAsianCuisine')   &
                            (locationsSampleStatsDF.third_pop_cuisine_nm != 'southAsianCuisine')]

print(locationsSampleStatsDF['third_pop_cuisine_nm'].value_counts(),  end="\n\n")
print('Total Num of rows in Filtered Sample file ............= {} '.format(locationsSampleStatsDF.shape[0] ) )  

eastAsianCuisine         464
southAmericanCuisine     346
southernEuroCuisine      220
northAmericanCuisine     172
caribbeanCuisine         155
southEastAsianCuisine     17
centralEuroCuisine         8
westernEuroCuisine         7
Name: third_pop_cuisine_nm, dtype: int64

Total Num of rows in Filtered Sample file ............= 1389 


In [15]:
locationsSampleStatsDF.replace(-666666666, np.nan,inplace=True)
locationsSampleStatsDF['region_cnt'] = locationsSampleStatsDF['region_cnt'].astype(float) 

ints     = ['int64']
intDF    = locationsSampleStatsDF.select_dtypes(include=ints)

categ     = ['object']
categDF   = locationsSampleStatsDF.select_dtypes(include=categ)
type(categDF)
########################################################################################################################################################
###
### Performing  Data normalization for Numeric Features
###
########################################################################################################################################################
floats  = ['float64']
floatDF = locationsSampleStatsDF.select_dtypes(include=floats)

floatDF  = floatDF.fillna(floatDF.median())

floatDF  = floatDF.apply(lambda x : (x - x.mean()) / (x.std() )) 

In [16]:
from sklearn.metrics          import classification_report, roc_auc_score, confusion_matrix
from sklearn.model_selection  import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.svm              import SVC
from sklearn.naive_bayes      import GaussianNB
from xgboost                  import XGBClassifier
from sklearn.neighbors        import KNeighborsClassifier
from sklearn.tree             import DecisionTreeClassifier
from sklearn.ensemble         import RandomForestClassifier 

from sklearn.pipeline         import Pipeline
from sklearn                  import metrics

from sklearn                  import preprocessing
from collections              import Counter

In [17]:
### I am adding locationName so that I can map them later for accuracy
x     = pd.concat([categDF['locationsName'],floatDF], axis=1)

y_clf = categDF[['third_pop_cuisine_nm']]    ## Target Variable for Predicting Preferred Cuisine 
y_reg = intDF[['third_pop_cuisine_cnt']]     ## Target Variable for Predicting How many restaurants can support in the region

In [18]:
### Creating training and testing sets for Classification
x_train, x_test, y_train, y_test = train_test_split(x,y_clf, test_size=.3, random_state=21)
print(x_train.shape)
print(y_train.shape)

x_train_locationNames = x_train['locationsName']
x_test_locationNames  = x_test['locationsName']

x_train = x_train.drop('locationsName',1)
x_test  = x_test.drop('locationsName',1)

x_train = x_train.values
y_train = y_train.values.ravel()

x_test= x_test.values
y_test= y_test.values.ravel()


##########################################################################################################################
# Transforming non numerical labels into numerical labels
##########################################################################################################################
encoder = preprocessing.LabelEncoder()

# encoding train and test labels 
y_clf_encoded   = encoder.fit_transform(y_clf)
y_train_encoded = encoder.transform(y_train)
y_test_encoded  = encoder.transform(y_test)

print("\nResampled Dataset Targert variable dist.............-> : \n{}".format(Counter(y_train_encoded)))  
print("\nResampled Dataset Targert variable dist.............-> : \n{}".format(Counter(y_test_encoded)))  

(972, 57)
(972, 1)

Resampled Dataset Targert variable dist.............-> : 
Counter({2: 324, 4: 242, 6: 158, 3: 119, 0: 109, 5: 10, 7: 6, 1: 4})

Resampled Dataset Targert variable dist.............-> : 
Counter({2: 140, 4: 104, 6: 62, 3: 53, 0: 46, 5: 7, 1: 4, 7: 1})


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
v_scoring='recall_micro'
v_cv     = 4
v_verbose= 1
jobs     = -1   ### n_jobs = -1 means maximum possible available
##########################################################################################################################
### 
### Model 1 : Decision Tree Classifier
###
##########################################################################################################################
# Set grid search params
grid_params = {'dt_clf__criterion'         : ["gini", "entropy"],
                'dt_clf__min_samples_split': [2,3,5],
                'dt_clf__max_depth'        : [None,15,20,25] , 
                'dt_clf__min_samples_leaf' : [1,5, 10, 20],
                'dt_clf__max_leaf_nodes'   : [None, 5, 10, 20]}

# Construct pipeline
pipe = Pipeline([('dt_clf', DecisionTreeClassifier(random_state=42))])

# Construct grid search
gs_dt = GridSearchCV(estimator=pipe, param_grid=grid_params, scoring=v_scoring ,cv=v_cv, verbose=v_verbose, n_jobs=jobs)
##########################################################################################################################
### 
### Model 2 : Random Forest Classifier
###
##########################################################################################################################
# Set grid search params                              
grid_params = [{'rf_clf__max_features'     : ['auto','sqrt'],
                'rf_clf__n_estimators'     : [1,10,20,50],               
                'rf_clf__min_samples_leaf' : [1,10,50],
                'rf_clf__max_depth'        : [5,10,20],        
                'rf_clf__min_samples_split': [2,4,8],
                'rf_clf__max_leaf_nodes'   : [10, 20]}]

# Construct pipeline
pipe = Pipeline([('rf_clf', RandomForestClassifier(random_state=42))])

# Construct grid search
gs_rf = GridSearchCV(estimator=pipe, param_grid=grid_params, scoring=v_scoring ,cv=v_cv, verbose=v_verbose, n_jobs=jobs)

##########################################################################################################################
### 
### Model 3 :  Support Vector Machines Classifier
###
##########################################################################################################################
# Set grid search params
param_range = [5,10,15,20]
grid_params = [{'svm_clf__kernel': ['linear', 'rbf'], 
                'svm_clf__C'     : [2, 4, 6, 8, 10],
                'svm_clf__gamma' : ['auto'] }]
              
# Construct pipeline
pipe = Pipeline([('svm_clf', SVC(random_state=42, probability = True))])

# Construct grid search
gs_svm = GridSearchCV(estimator=pipe, param_grid=grid_params, scoring=v_scoring ,cv=v_cv, verbose=v_verbose,n_jobs=jobs)

##########################################################################################################################
# List of pipelines for ease of iteration
grids = [gs_dt,
         gs_rf, 
         gs_svm]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'DT -Decision-Tree',
             1: 'RF -Random-Forst',
             2: 'SVM-Support-Vector-Machine'}  

In [20]:
best_acc = 0.0
best_clf = 0.0
best_gs  = ''

for idx, gsm in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx][4:])    
    #####################################################################################################
    ### Fit training set using grid search    #print(gsm.get_params().keys())
    #####################################################################################################
    #gsm.fit(x_train_res,y_train_res.ravel())
    
    gsm.fit(x_train,y_train_encoded)
    #####################################################################################################
    ### Printing 
    ###  1. Best score on training set
    ###  2. Best params on training set
    ###  3. Classification report and Accuracy on test set
    #####################################################################################################
    # Best Params
    #print('Best params..............-> : {}'.format(gsm.best_params_))    

    # Best Recall 
    print('Best Recall..............-> : %.3f' % gsm.best_score_)

    # Predicting on Best Model 
    y_pred = gsm.predict(x_test)
    y_pred_label = list(encoder.inverse_transform(y_pred))  
                              
    # Print Classification Report and Accuracy
    print("Test Data Accuracy ......-> {}".format(metrics.accuracy_score(y_test, y_pred_label)))    
    print("\n")    
    
    print(confusion_matrix(y_test,y_pred_label))    
    print("\n")
    
    print(classification_report(y_test, y_pred_label))


Estimator: Decision-Tree
Fitting 4 folds for each of 384 candidates, totalling 1536 fits


[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1536 out of 1536 | elapsed:   24.0s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Best Recall..............-> : 0.401
Test Data Accuracy ......-> 0.434052757793765


[[ 22   0  17   0   7   0   0   0]
 [  2   0   2   0   0   0   0   0]
 [  6   0 133   0   1   0   0   0]
 [  5   0  25   0  23   0   0   0]
 [ 14   0  64   0  26   0   0   0]
 [  1   0   6   0   0   0   0   0]
 [  2   0  60   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]]


                       precision    recall  f1-score   support

     caribbeanCuisine       0.42      0.48      0.44        46
   centralEuroCuisine       0.00      0.00      0.00         4
     eastAsianCuisine       0.43      0.95      0.60       140
 northAmericanCuisine       0.00      0.00      0.00        53
 southAmericanCuisine       0.46      0.25      0.32       104
southEastAsianCuisine       0.00      0.00      0.00         7
  southernEuroCuisine       0.00      0.00      0.00        62
   westernEuroCuisine       0.00      0.00      0.00         1

          avg / total       0.30      0.43      0.33       417



[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 898 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed:   45.0s finished


Best Recall..............-> : 0.386
Test Data Accuracy ......-> 0.4028776978417266


[[  6   0  23   1  16   0   0   0]
 [  0   0   3   0   1   0   0   0]
 [  3   0 123   0  14   0   0   0]
 [ 10   0  21   1  18   0   3   0]
 [  3   0  64   0  37   0   0   0]
 [  0   0   6   0   1   0   0   0]
 [  2   0  57   0   2   0   1   0]
 [  0   0   1   0   0   0   0   0]]


                       precision    recall  f1-score   support

     caribbeanCuisine       0.25      0.13      0.17        46
   centralEuroCuisine       0.00      0.00      0.00         4
     eastAsianCuisine       0.41      0.88      0.56       140
 northAmericanCuisine       0.50      0.02      0.04        53
 southAmericanCuisine       0.42      0.36      0.38       104
southEastAsianCuisine       0.00      0.00      0.00         7
  southernEuroCuisine       0.25      0.02      0.03        62
   westernEuroCuisine       0.00      0.00      0.00         1

          avg / total       0.37      0.40      0.31       417


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   16.2s finished


Best Recall..............-> : 0.388
Test Data Accuracy ......-> 0.434052757793765


[[ 10   0  12   0  24   0   0   0]
 [  1   0   2   0   1   0   0   0]
 [  4   0 121   0  15   0   0   0]
 [  8   0  19   4  22   0   0   0]
 [  7   0  51   0  46   0   0   0]
 [  0   0   6   0   1   0   0   0]
 [  1   0  55   0   6   0   0   0]
 [  0   0   1   0   0   0   0   0]]


                       precision    recall  f1-score   support

     caribbeanCuisine       0.32      0.22      0.26        46
   centralEuroCuisine       0.00      0.00      0.00         4
     eastAsianCuisine       0.45      0.86      0.59       140
 northAmericanCuisine       1.00      0.08      0.14        53
 southAmericanCuisine       0.40      0.44      0.42       104
southEastAsianCuisine       0.00      0.00      0.00         7
  southernEuroCuisine       0.00      0.00      0.00        62
   westernEuroCuisine       0.00      0.00      0.00         1

          avg / total       0.41      0.43      0.35       417



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### <font color = brown><i> Analysis of Results & Discussion</font></i> <a name="results"></a>
<font face=calibri size=3> From the above 3 Machine Learning Models, I can see all 3 classificagtion models are able to predict 'eastAsianCuisine' with good f1-score i.e. more about .60

Since Data is highly imbalanced, minority classes tend to be predicted as majority classes. So we might need more data set to find that correctly.


Nex stage of analysis is to find the number of Restaurants regions can support on basis of regression. The regions which has maximum potential will be the one which has highest difference between Predicted count and Existing count.


<br/><br/>
</font>
[Back To ToC](#ToC)

### <font color = brown><i> Conclusion & Further Study</font></i> <a name="conclusion"></a>
<font face=calibri size=3> 
<br/><br/>

In this study I hypothesized how Demographic information of regions and along with exisiting restautants, we can go about finding what type of Cusisine and what number of additional restaurants can be set in a regions as defined by US census.


Due to limits on API usage I could do this analysis on limited amount of Data. But would do this study on larger dataset and predicti What is the maximum revenue one can derive by setting restaurant in each region.

<br/>

This conclides this submission the project. 

</font>
[Back To ToC](#ToC)